<a href="https://colab.research.google.com/github/GoldenSnow-Xue/schulich_data_science/blob/main/Class_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163494 sha256=170bde0026539ac72c0fac74655c37066a5faf2e0ce891e937d469423377a01d
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
import pandas as pd
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import cross_validate

In [4]:
ratings = pd.read_csv("https://raw.githubusercontent.com/GoldenSnow-Xue/Schulich_Data_Science_Dataset/main/ratings.csv")
movies = pd.read_csv("https://raw.githubusercontent.com/GoldenSnow-Xue/Schulich_Data_Science_Dataset/main/movies.csv")

In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [6]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
data = pd.merge(ratings, movies, on="movieId")


In [8]:
data.describe()

,userId,movieId,rating,timestamp
count,105339.000000,105339.000000,105339.000000,1.053390e+05
mean,364.924539,13381.312477,3.516850,1.130424e+09
std,197.486905,26170.456869,1.044872,1.802660e+08
min,1.000000,1.000000,0.500000,8.285650e+08
25%,192.000000,1073.000000,3.000000,9.711008e+08
50%,383.000000,2497.000000,3.500000,1.115154e+09
75%,557.000000,5991.000000,4.000000,1.275496e+09
max,668.000000,149532.000000,5.000000,1.452405e+09


In [11]:
reader = Reader(rating_scale=(0.5,5))

In [12]:
data = Dataset.load_from_df(data[['userId', 'movieId','rating']], reader)

In [13]:
data

In [14]:
params = {'name':'cosine', 'user_based':False}

In [15]:
model = KNNBasic(sim_options=params)

In [16]:
cross_validate(model, data, measures=['RMSE','MAE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9710  0.9728  0.9713  0.9769  0.9786  0.9741  0.0031  
MAE (testset)     0.7554  0.7550  0.7537  0.7615  0.7585  0.7568  0.0028  
Fit time          12.28   10.26   10.67   10.70   9.65    10.71   0.87    
Test time         12.93   14.40   13.03   12.99   13.47   13.36   0.55    


{'test_rmse': array([0.97100327, 0.97280775, 0.97127439, 0.97693971, 0.97860964]),
 'test_mae': array([0.75538207, 0.75503742, 0.75373439, 0.76145839, 0.75848207]),
 'fit_time': (12.28172755241394,
  10.256772518157959,
  10.668715476989746,
  10.695377826690674,
  9.650819778442383),
 'test_time': (12.934321165084839,
  14.404011011123657,
  13.02695083618164,
  12.985633850097656,
  13.467002630233765)}

In [17]:
from surprise import accuracy
from surprise.model_selection import train_test_split

In [19]:
trainset, testset = train_test_split(data, test_size=0.2)

In [37]:
model2 = KNNBasic(sim_options={'user_based':True})
model2.fit(trainset)
predictions = model2.test(testset)
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9580


0.9579653284019307

In [37]:
model2 = KNNBasic(min_k = 5, k=10, sim_options={'name':'msd','user_based':True})
model2.fit(trainset)
predictions = model2.test(testset)
accuracy.rmse(predictions)

In [38]:
model2 = KNNBasic(min_k = 5, k=10, sim_options={'name':'cosine','user_based':True})
model2.fit(trainset)
predictions = model2.test(testset)
accuracy.rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9905


0.9904692646887234

In [39]:
model2.predict(668, 16)

Prediction(uid=668, iid=16, r_ui=None, est=4.049896766110635, details={'actual_k': 10, 'was_impossible': False})

if change k = 100, then actual_k = 100

In [40]:
movies[movies['movieId']==16]

,movieId,title,genres
15,16,Casino (1995),Crime|Drama


In [41]:
final_dataset = pd.merge(movies, ratings, on='movieId')

In [42]:
final_dataset['predicted_rating'] = final_dataset.apply(lambda x:model2.predict(x['userId'],x['movieId']).est, axis=1)

In [43]:
final_dataset.head()

,movieId,title,genres,userId,rating,timestamp,predicted_rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2,5.0,859046895,4.10000
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,1303501039,4.10000
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,5.0,858610933,3.95000
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.0,850815810,4.09935
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14,4.0,851766286,4.00000


In [44]:
from surprise import SVD

In [46]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8750  0.8681  0.8659  0.8687  0.8695  0.8694  0.0030  
Fit time          1.99    2.07    2.99    1.97    2.09    2.22    0.39    
Test time         0.13    0.51    0.18    0.39    0.14    0.27    0.15    


{'test_rmse': array([0.87498098, 0.86814969, 0.86588976, 0.86870094, 0.86948216]),
 'fit_time': (1.9909052848815918,
  2.066345453262329,
  2.9916670322418213,
  1.973438024520874,
  2.0859696865081787),
 'test_time': (0.13182520866394043,
  0.5147020816802979,
  0.18436574935913086,
  0.3872721195220947,
  0.1417834758758545)}

In [47]:
trainset = data.build_full_trainset()

In [48]:
svd.fit(trainset)

In [49]:
final_dataset['svd_pred'] = final_dataset.apply(lambda x: svd.predict(x['userId'],x['movieId']).est, axis=1)

In [50]:
final_dataset.head()

,movieId,title,genres,userId,rating,timestamp,predicted_rating,svd_pred
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2,5.0,859046895,4.10000,4.478465
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,1303501039,4.10000,3.471472
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,5.0,858610933,3.95000,4.175733
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.0,850815810,4.09935,4.006759
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14,4.0,851766286,4.00000,3.972182
